
---

# 📊 Proyecto Final: Inteligencia de Negocios y Minería de Datos

## 📘 Informe de Proyecto Final

**Materia:** Inteligencia de Negocios (ICC-321-T)
**Tema:** Dashboard Interactivo y Modelo de Minería de Datos Descriptivo

**Autores:**

* Randy Alexander Germosén Ureña *(1013-4707)*
* Fernando Almonte Delgado *(1015-7628)*

**Repositorio:**
[icc321-2025-final](https://github.com/TZeik/icc321-2025-final) <img src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" width="15" height="15"/>

---

## 🎯 Objetivo del Proyecto

El propósito de este proyecto es desarrollar una solución integral de Inteligencia de Negocios utilizando datos públicos del gobierno de la República Dominicana. Consta de dos componentes principales:

1. **Dashboard Interactivo:** Permite visualizar y monitorear métricas de gasto y nómina para apoyar la toma de decisiones.
2. **Modelo de Minería de Datos:** Implementación de un modelo descriptivo (Clustering) para descubrir patrones y segmentar perfiles de empleados.

---

## 📂 Datasets Utilizados

Se procesaron y unificaron datos históricos abarcando el periodo **2018–2025**:

1. **Nómina de la Contraloría General de la República:**
   Información detallada sobre empleados, cargos, departamentos y sueldos.
2. **Índice de Precios al Consumidor (IPC):**
   Datos del Banco Central utilizados para calcular el **salario real** (ajustado por inflación) en comparación con el salario nominal.

---

## 🧠 Metodología

El desarrollo del proyecto se estructuró en las siguientes fases técnicas:

### 1. Ingeniería de Datos (ETL)

* **Extracción y Limpieza:**

  * Unificación de múltiples archivos CSV mensuales/anuales.
  * Estandarización de nombres de cargos, normalización de formatos monetarios y corrección de codificación (`latin-1`, `utf-8`).
  * Homogeneización de los nombres de los meses.
* **Enriquecimiento:**

  * Cruce entre nómina e IPC para calcular la pérdida de poder adquisitivo.

### 2. Almacenamiento (Data Warehousing)

* Implementación de un **Data Warehouse** local con **SQLite**.
* Diseño bajo un **Esquema en Estrella**, con:

  * Tabla de hechos: `fact_nomina`
  * Tablas de dimensiones: `dim_empleado`, `dim_tiempo`

### 3. Visualización (Dashboard)

* Creación de Dashboard interactivo en **Tableau Public**.
* Diseño de KPIs como:

  * Gasto total,
  * Brecha salarial,
  * Evolución de plantilla,
  * Tendencias del salario real vs nominal.

### 4. Minería de Datos (Machine Learning)

* **Preprocesamiento:**
  Codificación de variables categóricas y escalado numérico.
* **Modelado:**
  Aplicación de **K-Means Clustering** para identificar grupos de empleados con características similares.
* **Evaluación:**

  * Método del Codo
  * Coeficiente de Silueta

---

## 📊 Resultados Principales

La solución permite analizar hallazgos relevantes como:

* Diferencias entre **Sueldo Nominal** y **Sueldo Real** a lo largo del tiempo.
* Identificación de departamentos con mayor incremento en el gasto de nómina.
* Clusters de empleados basados en sueldo, cargo y antigüedad, revelando patrones ocultos en la organización.

---

## 🧩 Herramientas Utilizadas

### Lenguajes y Entorno

* **Python 3.x** (Jupyter Notebook)

### Librerías Principales

* `pandas` — Manipulación y limpieza de datos
* `sqlite3` — Data Warehouse local
* `scikit-learn` — Algoritmo K-Means y métricas
* `matplotlib` — Visualización del método del codo

### Visualización

* **Tableau Public** — Dashboard interactivo final

---



In [1]:
import pandas as pd
import sqlite3
import numpy as np

### Configuración de Entorno y Reglas de Negocio
Importación de librerías y definición de funciones auxiliares para la limpieza de datos monetarios, estandarización de texto y cálculo estimado del Impuesto Sobre la Renta (ISR).

In [2]:
meses_map = {
    'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 'MAYO': 5, 'JUNIO': 6,
    'JULIO': 7, 'AGOSTO': 8, 'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12
}

meses_inv_map = {v: k for k, v in meses_map.items()}

def limpiar_moneda(valor):
    if pd.isna(valor): return 0.0
    s = str(valor).replace('RD$', '').replace(',', '').strip()
    try:
        return float(s)
    except:
        return 0.0

def limpiar_texto(texto):
    if pd.isna(texto): return "DESCONOCIDO"
    return str(texto).strip().upper()

def calcular_isr_estimado(sueldo_mensual):
    """
    Cálculo simplificado del ISR mensual (Escala aproximada DGII Rep. Dom).
    Exento hasta ~34,685.
    """
    if sueldo_mensual <= 34685:
        return 0.0
    elif sueldo_mensual <= 52027:
        excedente = sueldo_mensual - 34685
        return excedente * 0.15
    elif sueldo_mensual <= 72260:
        excedente = sueldo_mensual - 52027
        return 2601 + (excedente * 0.20)
    else:
        excedente = sueldo_mensual - 72260
        return 6648 + (excedente * 0.25)

print("Funciones y lógica de impuestos configuradas.")

Funciones y lógica de impuestos configuradas.


### Extracción, Transformación y Limpieza (ETL)
Carga de los datasets crudos (IPC y Nómina), normalización de nombres de columnas, conversión de tipos de datos y filtrado de registros nulos o inválidos para asegurar la calidad de los datos base.

In [3]:
# CARGA DE IPC
df_ipc = pd.read_csv('./raw_data/ipc_base_1984-2025.csv', sep=';')
df_ipc = df_ipc[df_ipc['PERIODO'] >= 2018].copy()
df_ipc['MES_LIMPIO'] = df_ipc['MES'].apply(limpiar_texto).map(meses_map)

# Seleccionamos columnas clave para la dimensión tiempo
df_ipc_clean = df_ipc[['PERIODO', 'MES_LIMPIO', 'INDICE']].rename(columns={
    'PERIODO': 'ANIO', 'MES_LIMPIO': 'MES_NUM', 'INDICE': 'IPC'
})

# CARGA DE NÓMINA
df_nomina = pd.read_csv('./raw_data/nomina-empleados-fijos-y-contratados-CSV-2018-2025.csv', 
                        sep=';', encoding='latin-1', low_memory=False)

# Normalización de columnas
df_nomina.columns = (df_nomina.columns.str.strip().str.upper()
                     .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8"))
df_nomina.columns = [c.replace('AO', 'ANIO') for c in df_nomina.columns]

# Limpiezas básicas
df_nomina['SUELDO_NOMINAL'] = df_nomina['SUELDO'].apply(limpiar_moneda)
df_nomina['ANIO'] = pd.to_numeric(df_nomina['ANIO'].astype(str).str.extract(r'(\d{4})')[0], errors='coerce')
df_nomina['MES_NUM'] = df_nomina['MES'].astype(str).apply(limpiar_texto).map(meses_map)

# Limpieza de textos
for col in ['NOMBRE', 'FUNCION', 'DEPARTAMENTO', 'ESTATUS']:
    df_nomina[col] = df_nomina[col].apply(limpiar_texto)

# Correcciones específicas de caracteres
df_nomina['NOMBRE'] = df_nomina['NOMBRE'].str.replace('Ï¿½', 'Ñ', regex=False)
df_nomina['DEPARTAMENTO'] = df_nomina['DEPARTAMENTO'].str.replace('AUDITORÏ¿½A', 'AUDITORIA', regex=False)

# Eliminar nulos críticos
df_nomina.dropna(subset=['ANIO', 'MES_NUM', 'SUELDO_NOMINAL'], inplace=True)
df_nomina['ANIO'] = df_nomina['ANIO'].astype(int)
df_nomina['MES_NUM'] = df_nomina['MES_NUM'].astype(int)

print(f"Datos base cargados. Registros en nómina: {len(df_nomina)}")

Datos base cargados. Registros en nómina: 152728


### Construcción del Data Warehouse (Esquema en Estrella)
Transformación de la tabla plana en un modelo dimensional: se generan las tablas de dimensiones (`DIM_TIEMPO`, `DIM_INSTITUCION`, `DIM_EMPLEADO`) con sus llaves primarias y se construye la tabla de hechos (`FACT_NOMINA`) con las métricas de negocio y llaves foráneas.

In [4]:
print("Iniciando construcción del Data Warehouse...")

# ---------------------------------------------------------
# CREAR DIMENSIÓN TIEMPO (DIM_TIEMPO)
# ---------------------------------------------------------
dim_tiempo = df_ipc_clean[['ANIO', 'MES_NUM', 'IPC']].drop_duplicates().copy()
dim_tiempo['mes_nombre'] = dim_tiempo['MES_NUM'].map(meses_inv_map)

# Generar PK
dim_tiempo.reset_index(drop=True, inplace=True)
dim_tiempo['id_tiempo'] = dim_tiempo.index + 1

# Renombrar
dim_tiempo = dim_tiempo.rename(columns={'ANIO': 'año', 'MES_NUM': 'mes_numero', 'IPC': 'valor_ipc'})
dim_tiempo = dim_tiempo[['id_tiempo', 'año', 'mes_nombre', 'mes_numero', 'valor_ipc']]
print(f"1. DIM_TIEMPO creada ({len(dim_tiempo)} registros).")


# ---------------------------------------------------------
# CREAR DIMENSIÓN INSTITUCIÓN (DIM_INSTITUCION)
# ---------------------------------------------------------
dim_institucion = df_nomina[['DEPARTAMENTO', 'FUNCION']].drop_duplicates().copy()
dim_institucion.reset_index(drop=True, inplace=True)
dim_institucion['id_institucion'] = dim_institucion.index + 1

# Lógica Grupo Ocupacional
def asignar_grupo(cargo):
    cargo = str(cargo)
    if any(x in cargo for x in ['DIRECTOR', 'ENCARGADO', 'GERENTE']): return 'ESTRATEGICO'
    if any(x in cargo for x in ['ANALISTA', 'COORDINADOR', 'SUPERVISOR', 'ABOGADO', 'AUDITOR']): return 'PROFESIONAL'
    if any(x in cargo for x in ['TECNICO', 'SOPORTE']): return 'TECNICO'
    return 'OPERATIVO'

dim_institucion['grupo_ocupacional'] = dim_institucion['FUNCION'].apply(asignar_grupo)

# Renombrar
dim_institucion = dim_institucion.rename(columns={'DEPARTAMENTO': 'departamento', 'FUNCION': 'cargo_normalizado'})
print(f"2. DIM_INSTITUCION creada ({len(dim_institucion)} cargos únicos).")


# ---------------------------------------------------------
# CREAR DIMENSIÓN EMPLEADO (DIM_EMPLEADO)
# ---------------------------------------------------------
dim_empleado = df_nomina[['NOMBRE', 'ESTATUS']].drop_duplicates(subset=['NOMBRE'], keep='last').copy()

dim_empleado.reset_index(drop=True, inplace=True)
dim_empleado['id_empleado'] = dim_empleado.index + 1

# Renombrar
dim_empleado = dim_empleado.rename(columns={'NOMBRE': 'nombre_completo', 'ESTATUS': 'estatus_laboral'})

# Reordenar columnas (Solo ID, Nombre y Estatus)
dim_empleado = dim_empleado[['id_empleado', 'nombre_completo', 'estatus_laboral']]

print(f"3. DIM_EMPLEADO creada ({len(dim_empleado)} empleados únicos).")


# ---------------------------------------------------------
# CREAR TABLA DE HECHOS (FACT_NOMINA)
# ---------------------------------------------------------
# 1. Unir con Tiempo
fact_base = pd.merge(df_nomina, dim_tiempo, left_on=['ANIO', 'MES_NUM'], right_on=['año', 'mes_numero'], how='inner')

# 2. Unir con Institución
fact_base = pd.merge(fact_base, dim_institucion, left_on=['DEPARTAMENTO', 'FUNCION'], right_on=['departamento', 'cargo_normalizado'], how='inner')

# 3. Unir con Empleado
fact_base = pd.merge(fact_base, dim_empleado, left_on=['NOMBRE'], right_on=['nombre_completo'], how='inner')

# Métricas
ipc_base_val = dim_tiempo['valor_ipc'].min()
fact_base['sueldo_real'] = (fact_base['SUELDO_NOMINAL'] / fact_base['valor_ipc']) * ipc_base_val
fact_base['monto_impuestos'] = fact_base['SUELDO_NOMINAL'].apply(calcular_isr_estimado)

# Selección Final
fact_nomina = fact_base[['id_empleado', 'id_tiempo', 'id_institucion', 
                         'SUELDO_NOMINAL', 'sueldo_real', 'monto_impuestos']].copy()

fact_nomina.rename(columns={'SUELDO_NOMINAL': 'sueldo_nominal'}, inplace=True)
fact_nomina.reset_index(drop=True, inplace=True)
fact_nomina.index.name = 'id_fact'
fact_nomina = fact_nomina.reset_index()

print(f"4. FACT_NOMINA creada ({len(fact_nomina)} registros de hechos).")

Iniciando construcción del Data Warehouse...
1. DIM_TIEMPO creada (94 registros).
2. DIM_INSTITUCION creada (4352 cargos únicos).
3. DIM_EMPLEADO creada (3786 empleados únicos).
4. FACT_NOMINA creada (152728 registros de hechos).


### Almacenamiento y Exportación Final
Persistencia del modelo dimensional en una base de datos local SQLite y exportación de las tablas individuales a formato CSV para su visualización en Tableau.

In [5]:
# Conexión a Base de Datos
db_name = 'DW_Nomina_Publica.db'
conn = sqlite3.connect(db_name)

# Guardar en SQLite
dim_tiempo.to_sql('DIM_TIEMPO', conn, if_exists='replace', index=False)
dim_empleado.to_sql('DIM_EMPLEADO', conn, if_exists='replace', index=False)
dim_institucion.to_sql('DIM_INSTITUCION', conn, if_exists='replace', index=False)
fact_nomina.to_sql('FACT_NOMINA', conn, if_exists='replace', index=False)

conn.close()

# Exportar CSVs para Tableau
dim_tiempo.to_csv('DIM_TIEMPO.csv', index=False, encoding='utf-8')
dim_empleado.to_csv('DIM_EMPLEADO.csv', index=False, encoding='utf-8')
dim_institucion.to_csv('DIM_INSTITUCION.csv', index=False, encoding='utf-8')
fact_nomina.to_csv('FACT_NOMINA.csv', index=False, encoding='utf-8')

print(" Creación de DW completada. ")

 Creación de DW completada. 
